In [1]:
#importing necessary libraries
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

import time, sys
import tqdm.notebook as tqdm
import pandas as pd
import numpy as np

import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [2]:
#imorting the website links from where we will scrape data
url1 = "https://www.cardekho.com/"
url2 = "https://www.olX.in/"
url3 = "https://www.cars24.com/"
url4 = "https://www.carwale.com/used-cars/"

In [3]:
page = requests.get(url1)
print("Response number from Cars 24 URL is:", page) # to show the response output from the webpage
soup = BeautifulSoup(page.content)

Response number from Cars 24 URL is: <Response [403]>


In [4]:
#connecting to chromedriver
driver = webdriver.Chrome(r'C:\Users\poonam\Downloads\chromedriver_win32 (1)\chromedriver.exe')

In [5]:
#opening website CARS24
driver.get(url1)
driver.maximize_window()

In [7]:
# Fetching urls to navigate location page
cities = driver.find_elements(By.XPATH,'//ul[@class="listing gsc_col-md-12 gsc_col-lg-12"]//li/a')

In [8]:
#creating empty list and grtting links for locations
city = []
for i in cities:
    city.append(i.get_attribute('href'))

In [9]:
city

['https://www.cardekho.com/used-cars+in+ahmedabad',
 'https://www.cardekho.com/used-cars+in+bangalore',
 'https://www.cardekho.com/used-cars+in+mumbai',
 'https://www.cardekho.com/used-cars+in+jaipur',
 'https://www.cardekho.com/used-cars+in+gurgaon',
 'https://www.cardekho.com/used-cars+in+delhi-ncr',
 'https://www.cardekho.com/used-cars+in+chennai',
 'https://www.cardekho.com/used-cars+in+pune',
 'https://www.cardekho.com/used-cars+in+hyderabad',
 'https://www.cardekho.com/used-cars+in+kolkata',
 'https://www.cardekho.com/used-cars+in+agra',
 'https://www.cardekho.com/used-cars+in+lucknow']

In [10]:
#creating empty lists for scraping data
Car_brand = []
car_price = []
kms_driven = []
fuel = []
transmission = []
city_url = []
car_urls = []

for i in city:
    driver.get(i)
    time.sleep(3)
    
    for _ in range(150):
        time.sleep(1)
        driver.execute_script("window.scrollBy(0,1000)")
    current_url = driver.current_url    
    car = driver.find_elements(By.XPATH,'//div[@class="gsc_col-xs-7  carsName"]/a')
    for i in car:
        car_urls.append(i.get_attribute('href'))        # saving all the car urls for further information collecting purposes   
        
    #fetching car brand and location urls of the used cars
    for i in driver.find_elements(By.XPATH,'//div[@class="gsc_col-xs-7  carsName"]/a'):
        Car_brand.append(i.text)
        city_url.append(current_url)
    
    #fetching price of the cars
    for i in driver.find_elements(By.XPATH,'//span[@class="amnt "]'):
        car_price.append(i.text)
    
    #fetching driven kilometers of the used cars
    for i in driver.find_elements(By.XPATH,'//div[@class="truncate dotlist-2"]//div[1]'):
        kms_driven.append(i.text)
    
    #fetching fuel type of the cars
    for i in driver.find_elements(By.XPATH,'//div[@class="truncate dotlist-2"]//div[2]'):
        fuel.append(i.text)
    
    #fetching transmission of the cars
    for i in driver.find_elements(By.XPATH,'//div[@class="truncate dotlist-2"]//div[3]'):
        transmission.append(i.text)

In [ ]:
#printing length of car features to confirm
print(len(car_urls),len(Car_brand), len(city_url), len(car_price), len(kms_driven), len(fuel), len(transmission))

In [15]:
#creating empty list to scrap the specification of the used cars
seats = []
engine_disp = []
milage = []
color = []
max_power = []
front_brake = []
rear_brake = []
cargo_vol = []
height = []
width = []
length = []
weight = []
insp_score = []
top_speed = []

In [16]:
#for loop to get car urls
for i in car_urls:
    driver.get(i)
    time.sleep(2)
    
    #clicking on view all specifications button to view more features of the cars
    try:
        view_more = driver.find_element(By.XPATH,'//*[text() = "View All Specifications" or text() = "View More"]')
        driver.execute_script("arguments[0].scrollIntoView();", view_more)
        driver.execute_script("arguments[0].click();", view_more)
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    #fetching engine displacement of the cars
    try:
        eng = driver.find_element(By.XPATH,'//*[text()="Engine"]/following-sibling::div') 
        engine_disp.append(eng.text.replace('CC',''))
    except NoSuchElementException:
        engine_disp.append('-')
    
    #fetching Mileage of the cars
    try:
        mil = driver.find_element(By.XPATH,'//*[text()="Mileage"]/following-sibling::div')
        milage.append(mil.text)
    except NoSuchElementException:
        milage.append('-')
        
    #fetching seating capacity of the cars
    try:
        sts = driver.find_element(By.XPATH,'//*[text()="Seating Capacity"]/following-sibling::div')
        seats.append(sts.text)
    except NoSuchElementException:
        seats.append('-')
        
    #fetching colour of the cars
    try:
        clr = driver.find_element(By.XPATH,'//*[text()="Color"]/following-sibling::div')
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
        
    #fetching Maximum power of the cars
    try:
        pwr = driver.find_element(By.XPATH,'//*[text()="Max Power"]/following-sibling::div')
        max_power.append(pwr.text.replace('bhp',''))
    except NoSuchElementException:
        max_power.append('-')
        
    #fetching Front brake type of the cars
    try:
        fbrk = driver.find_element(By.XPATH,'//*[text()="Front Brake Type"]/following-sibling::div')
        front_brake.append(fbrk.text)
    except NoSuchElementException:
        front_brake.append('-')
        
    #fetching rear brake type of the cars
    try:
        rbrk = driver.find_element(By.XPATH,'//*[text()="Rear Brake Type"]/following-sibling::div')
        rear_brake.append(rbrk.text)
    except NoSuchElementException:
        rear_brake.append('-')
        
    #fetching details of cargo volume of the cars
    try:
        vol = driver.find_element(By.XPATH,'//*[text()="Cargo Volumn"]/following-sibling::div')
        cargo_vol.append(vol.text)
    except NoSuchElementException:
        cargo_vol.append('-')
        
    #fetching top speed of the cars
    try:
        speed = driver.find_element(By.XPATH,'//*[text()="Top Speed"]/following-sibling::div')
        top_speed.append(speed.text)
    except NoSuchElementException:
        top_speed.append('-')
        
    #fetching height of the cars
    try:
        hgt = driver.find_element(By.XPATH,'//*[text()="Height"]/following-sibling::div')
        height.append(hgt.text.replace('mm',''))
    except NoSuchElementException:
        height.append('-')
        
    #fetching width of the cars
    try:
        wth = driver.find_element(By.XPATH,'//*[text()="Width"]/following-sibling::div')
        width.append(wth.text.replace('mm',''))
    except NoSuchElementException:
        width.append('-')
        
    #fetching length of the cars
    try:
        lth = driver.find_element(By.XPATH,'//*[text()="Length"]/following-sibling::div')
        length.append(lth.text.replace('mm',''))
    except NoSuchElementException:
        length.append('-')
        
    #fetching gross weight of the cars
    try:
        weigh = driver.find_element(By.XPATH,'//*[text()="Gross Weight"]/following-sibling::div')
        weight.append(weigh.text.replace('kg',''))
    except NoSuchElementException:
        weight.append('-')
        
    #fetching Inspection score of the cars
    try:
        insp = driver.find_element(By.XPATH,'//div[@class="inspectionRating right-side gsc_col-xs-5 text-right"]/span')
        insp_score.append(insp.text)
    except NoSuchElementException:
        insp_score.append('-')

In [ ]:
#printing lengths of all the features
print(len(seats),len(engine_disp),len(milage),len(color),len(max_power),len(front_brake),len(rear_brake),len(cargo_vol),len(height),len(width),len(length),l

In [17]:
#creating a dataframe of scraped data
data = list(zip(Car_brand,fuel,kms_driven,engine_disp,transmission,milage,seats,color,max_power,front_brake,rear_brake,cargo_vol,height,width,length,weight,insp_score,top_speed,city_url,car_price))
df = pd.DataFrame(data, columns = ["Car_Name","Fuel_type","Running_in_kms","Endine_disp", "Gear_transmission","Milage_in_km/ltr","Seating_cap","color","M

SyntaxError: EOL while scanning string literal (Temp/ipykernel_5368/2882321600.py, line 3)

In [ ]:
#saving the scraped data into excel file
df.to_excel("Used_Cars.xlsx")